In [4]:
### SAVING SEIZURES AS PICKLE FILES TO LEIF
import numpy as np
import pandas as pd
import json
import os
from os.path import join as ospj
from utils import *
import scipy as sc

# Loading CONFIG
with open('config.json','r') as f:
    CONFIG = json.load(f)
usr = CONFIG["paths"]["iEEG_USR"]
passpath = CONFIG["paths"]["iEEG_PWD"]
datapath = CONFIG["paths"]["RAW_DATA"]
figpath = CONFIG["paths"]["FIGURES"]
pt_table = pd.DataFrame(CONFIG["patients"]).sort_values('ptID')
rid_hup = pd.read_csv(ospj(datapath,'rid_hup.csv'))
pt_list = pt_table.ptID.to_numpy()
lf_pt_list = pt_list[pt_table.lf_stim==1]

np.random.seed(42)

In [5]:
lf_pt_list

array(['HUP224', 'HUP225', 'HUP230', 'HUP235', 'HUP238', 'HUP246',
       'HUP247', 'HUP249', 'HUP250', 'HUP253', 'HUP257', 'HUP263',
       'HUP266', 'HUP267', 'HUP273'], dtype=object)

In [2]:
# downsampling factor
def get_factor(fs,target=512):
    if fs%target != 0:
        print("FS not divisible by target, will perform \
              integer division and new fs may not match target")
    return fs // target
TARGET = 512

In [6]:
# load in stim seizure information
sz_info = pd.read_csv(ospj(datapath,'stim_seizure_information - Revised_seizure_annotation.csv'))

In [7]:
sz_info

,Patient,IEEGname,EEC,UEO,Other onset description,approximate,end,source,stim,stim_channels,EEC onset channels,UEO onset channels,Notes,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,HUP224,HUP224_phaseII,71156.5913,71156.58349,NaN,71156.59,71190.99,Will manual,1.0,LB1-LB2,"LB1,LB2,LB3,LB4,LB5,LC1,LC2,LC3,LC4,LA1,LA2,LA...",NaN,NaN,NaN,# Low frequency seizures,14.0,"plus 3 from redcap (HUP261, 263, 266)"
1,HUP224,HUP224_phaseII,202170.6825,202170.68250,NaN,202170.68,202243.03,Erin manual,0.0,NaN,NaN,"RB2,RB3,RC2",NaN,NaN,# High frequency seizures,31.0,plus ??
2,HUP224,HUP224_phaseII,227267.9559,227267.95590,NaN,227267.96,227350.65,Erin manual,0.0,NaN,NaN,"RB2,RB3,RC2",NaN,NaN,# Patients,34.0,"plus 3 from redcap (HUP261, 263, 266)"
3,HUP224,HUP224_phaseII,339135.7773,339143.64350,NaN,339135.86,339234.20,Erin manual,0.0,NaN,NaN,"RB2,RB3,RC2",NaN,NaN,NaN,NaN,NaN
4,HUP224,HUP224_phaseII,415736.8300,415751.71870,NaN,415736.98,415805.57,Erin manual,0.0,NaN,NaN,NaN,no channels,NaN,NaN,NaN,NaN


In [ ]:
# Iterate through each patient
for pt in lf_pt_list:
    print(f"Starting Seizure Preprocessing for {pt}")
    raw_datapath = ospj(datapath,pt)
    # load dataframe of seizure times
    seizure_times = sz_info[sz_info.Patient == pt,:]
    # load electrode information
    if not os.path.exists(ospj(raw_datapath, "electrode_localizations.csv")):
        hup_no = pt[3:]
        rid = rid_hup[rid_hup.hupsubjno == hup_no].record_id.to_numpy()[0]
        recon_path = ospj('/mnt','leif','littlab','data',
                        'Human_Data','CNT_iEEG_BIDS',
                        f'sub-RID0{rid}','derivatives','ieeg_recon',
                        'module3/')
        if not os.path.exists(recon_path):
            recon_path =  ospj('/mnt','leif','littlab','data',
                        'Human_Data','recon','BIDS_penn',
                        f'sub-RID0{rid}','derivatives','ieeg_recon',
                        'module3/')
        electrode_localizations = electrode_localization(recon_path,rid)
        electrode_localizations.to_csv(ospj(raw_datapath,"electrode_localizations.csv"))
    else:
        electrode_localizations = pd.read_csv(ospj(raw_datapath,"electrode_localizations.csv"))
    ch_names = electrode_localizations[(electrode_localizations['index'] == 2) | (electrode_localizations['index'] == 3)]["name"].to_numpy()

    # loading seizures
    if not os.path.exists(ospj(raw_datapath, "seizures")):
        os.mkdir(ospj(raw_datapath, "seizures"))
    
    # Iterate through each seizure in pre-defined pkl file
    for i_sz,row in seizure_times.iterrows():
        print(f"Saving seizure number: {i_sz}")
        seizure,fs = get_iEEG_data(usr,passpath,
                                    row.IEEGname,
                                    row.approximate_onset*1e6,
                                    row.end*1e6,
                                    ch_names,
                                    force_pull = True)
        factor = get_factor(fs,TARGET)
        fsd = fs//factor
        seizure_ds = pd.DataFrame(sc.signal.decimate(seizure.to_numpy(),FACTOR,axis=0),columns=ch_names)           
        seizure_ds.to_pickle(ospj(raw_datapath,"seizures",f"{fsd}_seizure_{i_sz}_stim_{row.stim}.pkl"))